In [1]:
import datasets
from datasets import load_dataset

dataset = load_dataset("nateraw/pascal-voc-2012", split="train")

Using custom data configuration nateraw___pascal-voc-2012-c68607404d4811ac
Reusing dataset parquet (/aiffel/.cache/huggingface/datasets/parquet/nateraw___pascal-voc-2012-c68607404d4811ac/0.0.0/9296ce43568b20d72ff8ff8ecbc821a16b68e9b8b7058805ef11f06e035f911a)


In [2]:
dataset

Dataset({
    features: ['image', 'mask'],
    num_rows: 2913
})

**(공통)문제 1. 이미지 세그멘테이션 모델을 만들고, 전체 코드와 분류 결과를 주피터 노트북 파일로 자신의 깃헙에 제출합니다. (3점)**</br>

- 이미지 데이터를 적절히 전처리 한 후 데이터 파이프라인을 설계합니다.
- 모델은 U-net으로 고정합니다. 단, 올바른 훈련 진행을 위해 원본에서 변경이 필요할 수 있습니다.
- <span style='color: #ff0000'> 텐서플로우 라이브러리에서 모델을 불러오지 않고, 직접 모델 학습 코드를 짜야 합니다. </span>
- Data augmentation 등 케창딥에서 배운 모든 방법을 사용해 최고 기록을 달성해보세요.

*데이터 전처리*

In [3]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# 데이터 복사 (원본 보존 목적); byte값만 가져오기
image_data = [obj['bytes'] for obj in dataset['image'] if 'bytes' in obj]
mask_data = [obj['bytes'] for obj in dataset['mask'] if 'bytes' in obj]


# 데이터 분리 (8:2 비율로 train, val)
train_imgs, val_imgs, train_masks, val_masks = train_test_split(image_data, mask_data, test_size=0.2, random_state=42)

# byte값을 tensor로 변환하기
def bytes_2_tensor(byte_list, channels=None):
    res = []
    
    # byte_list == train_imgs, val_imgs, train_masks, val_masks
    for val in tqdm(byte_list, desc=("byte to tensor changing...")):
        res.append(tf.image.decode_image(val, dtype=tf.float32, channels=channels))
    res = np.array(res)
    
    return res

# byte 값 가진 list들을 tensor 값 가진 list로 변환
train_imgs = bytes_2_tensor(train_imgs)
val_imgs = bytes_2_tensor(val_imgs)
train_masks = bytes_2_tensor(train_masks, 1)
val_masks = bytes_2_tensor(val_masks, 1)

# 훈련 데이터 정규화
def normalize(train_data, channels=None):
    # train_data는 byte고, image랑 mask
    res = []
    
    for val in tqdm(train_data, desc="train data normalizing.."):
        normalized = val / 255.0
        res.append(normalized)
    return res

# 훈렌 데이터 정규화 수행
train_imgs = normalize(train_imgs)
train_masks = normalize(train_masks, 1)

byte to tensor changing...: 100%|██████████| 2330/2330 [00:30<00:00, 75.81it/s] 
/tmp/ipykernel_293/2997905440.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(res)
train data normalizing..: 100%|██████████| 2330/2330 [00:01<00:00, 2092.17it/s]


모델 생성 (`U-net`)

In [29]:
from tensorflow.keras.layers import Input, Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Concatenate, Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization, Activation
from tensorflow.keras.models import Model

# 반복되는 convolution 레이어를 conv_block()으로 선언
def conv_block(inputs, n_filters):
    x = Conv2D(n_filters, 3, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(n_filters, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x

def encoder_block(inputs, n_filters):
    x = conv_block(inputs, n_filters)
    p = MaxPooling2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip, n_filters):
    x = Conv2DTranspose(n_filters, (2, 2), strides=2, padding='same')(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, n_filters)
    return x

# encoder, decoder를 사용한 unet()
def unet(input_shape, num_classes):
    inputs = tf.keras.Input(input_shape)
    
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)
    b = conv_block(p4, 1024)
    d1 = decoder_block(b, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    if n_classes == 1:
        activation = 'sigmoid'
    else:
        activation = 'softmax'

    outputs = Conv2D(n_classes, 1, padding='same', activation=activation)(d4)

    model = tf.keras.Model(inputs, outputs)
    return model

In [30]:
# ..
def dice_coefficient(y_true, y_pred):
    smooth = 1.0
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2.0 * intersection + smooth) / (union + smooth)
    return dice

# 모델 생성
# 입력 이미지 크기와 클래스 수 설정
input_shape = (375, 500, 3)
num_classes = 2
model = unet(input_shape, n_classes)

# 옵티마이저
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 92, 124, 256), (None, 93, 125, 256)]

**문제 2. 이미지 분류 학습 결과를 텐서보드로 시각화합니다. (1점)**
- tensorboard.dev를 사용해 텐서보드 결과를 공유할 수 있는 url 주소 링크를 자신의 깃헙 Main_Quest_2 폴더 내의 Readme.md 파일에 게시합니다.

**문제 3. 가장 성능이 좋은 모델에 CAM을 적용해, 분류결과에 대한 근거를 시각화하고 분석해보세요. (1점)**</br>

- 케창딥의 Calss Activation Map 코드를 참고해 세그멘테이션 모델에 테스트 이미지를 넣고 추론한 결과에 CAM heatmap을 뽑아 주피터 노트북 파일에 포함해 제출하세요.

**문제 구성**  
a. 1번 : 필수 문제입니다.  
b. 2번 / 3번 : 두 문제 중 하나를 선택해서 풀어주세요.  
  
**배점**  
a. 최대 점수는 5점입니다.  
b. 1번 문제 : 3점  
c. 2번 / 3번 문제 : 각 1점  
    i. 단 세개를 다 푸신 분께는 퍼실 채점 결과 2분을 선정해 쉐밸그투 세미나에서 풀이결과를 소개해 +2점을 받으실 수 있는 기회를 드립니다.

**제출 방법2**  
- 제출기한 : 17:30까지
- 깃헙 폴더 파일명 : Main_Quest2
- 주피터 노트북 파일 명 : 본인이름_Pascal_VOC image segmentation.ipynb
    - 제출 양식을 지키지 않으면 채점 대상에서 제외됩니다.